# 3 - Pré-processamento e transformações

Nessa fase, construiremos alguns modelos específicos para texto para então treiná-los;

In [1]:
# caminho para instalação do pacote mltoolkit, com metricas e gráficos personalizados
# !pip install git+ssh://git@github.com/flimao/mltoolkit

In [2]:
import pandas as pd
import numpy as np
from matplotlib import rcParams, rcParamsDefault, pyplot as plt
import seaborn as sns
from mltoolkit import metrics, plots, NLP
import spacy

rcParams.update(rcParamsDefault)
rcParams['figure.dpi'] = 120
rcParams['figure.figsize'] = (10, 8)

In [3]:
# !python -m spacy download pt_core_news_lg
# !python -m spacy download pt_core_news_md
# !python -m spacy download pt_core_news_sm
nlp = spacy.load("pt_core_news_lg")

## Importação dos dados

Primeiramente, importamos os dados e aplicamos as transformações utilizadas na fase anterior:

In [4]:
# não tocaremos no conjunto de submissão

tweets_raw = pd.read_csv(
    r'../data/Train3Classes.csv',
)

In [5]:
# trocar tipos para acelerar o processamento (menos espaço em memória)
# e ativar possíveis otimizações internas ao pandas para certos tipos
def mudar_tipos(df):
    df = df.copy()

    df['id'] = df['id'].astype('string')
    df['tweet_date'] = pd.to_datetime(df['tweet_date'])
    df['sentiment'] = df['sentiment'].astype('category')

    return df

def remover_duplicatas(df):
    df = df.copy()

    df = df.drop_duplicates(subset = 'id')

    return df

# o índice é o id, visto que não há repetidos
# vantagem: o índice é removido automaticamente quando separamos em base de treino e teste.
def setar_index(df):
    df = df.copy()

    df = df.set_index('id')

    return df

tweets_full = (tweets_raw
    .pipe(mudar_tipos)
    .pipe(remover_duplicatas)
    .pipe(setar_index)
)

## Pré-processamento de texto

Vamos então implementar o pré-processamento do texto da fase anterior (Análise Exploratória de Texto).

Primeiramente vamos importar as *stopwords*:

In [6]:
with open(r'../data/stopwords_alopes.txt', encoding = 'utf8') as stopword_list:
    lst = stopword_list.read().splitlines()

stopwords_alopes = set([ stopword.strip() for stopword in lst ])

# em uma análise de sentimento, não queremos remover palavras com conotação negativa
remover_stopwords = {
    'não', 
}

stopwords_alopes -= remover_stopwords

In [7]:
preprocessing_full = lambda s: NLP.preprocessing(s, preproc_funs_args = [
    NLP.remove_links,
    NLP.remove_hashtags,
    NLP.remove_mentions,
    NLP.remove_numbers,
    NLP.remove_special_caract,
    NLP.lowercase,
    #remove_punkt,
    #(remove_stopwords, dict(stopword_list = stopword_list_alopes)),
    (NLP.tokenize_remove_stopwords_get_radicals_spacy, dict(
        nlp = nlp,
        stopword_list = stopwords_alopes,
    )),
])

Vamos então aplicar esse pré-processamento a uma amostra da base de *tweets* (para podermos iterar rapidamente caso necessário). 

Em um momento posterior, treinaremos a base completa.

In [8]:
amostra_eda = 5000
radicais = tweets_full.sample(amostra_eda)['tweet_text'].apply(preprocessing_full)

tweets = tweets_full.copy()
tweets['radicais'] = radicais
tweets = tweets[tweets.radicais.notna()]

In [9]:
tweets.sample(10)

,tweet_text,tweet_date,sentiment,query_used,radicais
id,,,,,
1045267908656541697,Pai é morto na frente da família após tentar p...,2018-09-27 11:04:24+00:00,2,g1,pai morto frente familia apo proteger filhar m...
1046961086115844096,Nao tenho nite e quero fechar help me :(,2018-10-02 03:12:29+00:00,0,:(,nao nite fechar help
1046227089647718401,com muito sono porém enrolando p dormir por sa...,2018-09-30 02:35:50+00:00,0,:(,sono pôr enrolar p dormir amanhar ir muito dor...
1046779663094349826,@kyokuga same tbh isso foi uma das cenas que m...,2018-10-01 15:11:34+00:00,0,:(,same tbh cena dar throw off tbh tambem ir almo...
1049124205709742082,vamos esclavas!!! :D #슈주_OneMoreTime_오늘오후6시 @S...,2018-10-08 02:27:57+00:00,1,:),ir esclavo d
1046761908517445632,"@rengafffffff Sim, é bem isso... :(",2018-10-01 14:01:01+00:00,0,:(,
1037062933690425344,Youtuber que mostrou luta contra doença morre ...,2018-09-04 19:40:45+00:00,2,exame,youtuber mostrar lutar doenca morrer ano
1049316346629185537,@SchuldinerSieg Puta merda! :( Pra que servem ...,2018-10-08 15:11:27+00:00,0,:(,puta merda pra servir desgracados atrasar bras...
1047545781157318656,peguei a armação nova de óculos e me achei hor...,2018-10-03 17:55:51+00:00,1,:),pegar armacao oculos achar horroroso


## Opções de modelos

Vamos agora olhar para alguns modelos que podemos utilizar.

Definiremos os modelos desejados, e então procederemos à comparação dos mesmos.

In [10]:
X = tweets['radicais']
y = tweets['sentiment']

In [11]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size = 0.3,
    stratify = y,
)

X_trains = {}
X_tests = {}

### 1. *Bag of Words* / `CountVectorizer`

*Bag of Words* é o processo onde traduzimos o texto já tratado para uma representação numérica que faça sentido para o modelo de *Machine Learning* consiga interpretá-lo.

In [12]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer()
X_trains['bow'] = cv.fit_transform(X_train).toarray()
X_tests['bow'] = cv.transform(X_test).toarray()

In [13]:
X_trains['bow']

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [14]:
X_trains['bow'].shape

(3500, 6318)

### 2. TF-IDF

***Term Frequency and Inverse Document Frequency*** é uma tranformação onde avaliamos a relevância das palavras pela **Frequência dos Termos** e multiplicamos pelo **Inverso da Frequência nos Documentos**.

Nesse contexto, um **documento** é cada um dos textos dentro de um *dataset*. Vamos entender cada um dos termos:

> **TF - Term Frequency**: é a frequência de vezes que um termo/palavra aparece em cada um dos documentos analisados (isso nos ajuda a avaliar a relevância daquela palavra);

> **IDF - Inverse Document Frequency**: aqui avaliamos em quantos documentos o termo/palavra aparece (dessa forma conseguimos entender a sua influência em identificar os textos);

In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(use_idf = True)

X_trains['tfidf'] = tfidf.fit_transform(X_train).todense()
X_tests['tfidf']  = tfidf.transform(X_test).todense()

In [16]:
X_trains['tfidf']

matrix([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]])

In [17]:
X_trains['tfidf'].shape

(3500, 6318)

### *Word2vec*

O *Word2vec* ([Wikipedia](https://en.wikipedia.org/wiki/Word2vec), [Gensim](https://radimrehurek.com/gensim/models/word2vec.html)) é uma rede neural onde associa-se vetores a cada palavra. Os vetores são tais que pretendem capturar as relações semânticas entre as mesmas.

Por exemplo, se tivermos em nosso vocabulário as palavras *rei*, *rainha*, *homem* e *mulher*, poderíamos fazer a seguinte operação vetorial:

$ \vec{v}_{rei} - \vec{v}_{homem} + \vec{v}_{mulher} = \vec{v}_{rainha}$

In [18]:
from gensim.models import Word2Vec

X_train_tokens = X_train.str.split(' ').to_list()
X_test_tokens = X_test.str.split(' ').to_list()

w2v_model = Word2Vec(
    sentences = X_train_tokens, 
    vector_size = 2,  # este parâmetro é o equivalente ao número de features. 
    min_count = 1, 
    workers = 2
)

In [19]:
w2v_model.wv.most_similar(positive = ['bolsonaro'], negative = ['haddad'])

[('apartamento', 1.0000001192092896),
 ('casal', 0.9999997615814209),
 ('enriquecimento', 0.9999997019767761),
 ('who', 0.9999993443489075),
 ('witzel', 0.9999993443489075),
 ('usp-sao', 0.9999991059303284),
 ('atua', 0.9999940991401672),
 ('tuite', 0.9999898076057434),
 ('risada', 0.9999882578849792),
 ('terreo', 0.9999875426292419)]

In [20]:
w2v_model.wv.similarity('bolsonaro', 'haddad')

0.9563616

In [21]:
# função para, dado um modelo word2vec e um conjunto de frases em formato de token (listas de listas), 
# construir os vetores associados a cada uma

def build_word2vec_vectors(model, phrases, vector_combination):

    X = []
    vector_size = model.vector_size

    for phrase in phrases:

        ntokens = len(phrase)
        vectors = np.zeros(shape = (ntokens, vector_size))

        for i, token in enumerate(phrase):
            try:

                vectors[i, :] = model.wv[token]
            except KeyError:  # token not present in corpus
                vectors[i, :] = 0

        X.append(vector_combination(vectors))
    
    return np.asarray(X)

# função para, dados conjuntos de frases de treino e teste, construir os vetores
# associados
def build_word2vec_model(
    X_train, X_test, 
    vector_combination,
    is_token = False,
    **kwargs
):
    # kwargs = arguments for Word2Vec class
    
    if is_token:
        X_train_tokens = X_train
        X_test_tokens = X_test
    else:
        X_train_tokens = X_train.str.split(' ').to_list()
        X_test_tokens = X_test.str.split(' ').to_list()
    
    # instantiate, build and train model
    w2v_model = Word2Vec(
        sentences = X_train_tokens, 
        **kwargs
    )

    # build vectors
    X_train_w2v = build_word2vec_vectors(
        model = w2v_model, 
        phrases = X_train_tokens, 
        vector_combination = vector_combination
    )

    X_test_w2v = build_word2vec_vectors(
        model = w2v_model, 
        phrases = X_test_tokens, 
        vector_combination = vector_combination
    )

    return w2v_model, X_train_w2v, X_test_w2v

In [22]:
w2v_model_sum, X_trains['word2vec_sum'], X_tests['word2vec_sum'] = build_word2vec_model(
    X_train, X_test, 
    is_token = False,
    # --- word2vec model parameters
    vector_size = 50, # este parâmetro é o equivalente ao número de features. 
    min_count = 2, workers = 2,
    # --- vector_combination
    vector_combination = lambda x: np.sum(x, axis = 0),
)

w2v_model_mean, X_trains['word2vec_mean'], X_tests['word2vec_mean'] = build_word2vec_model(
    X_train, X_test, 
    is_token = False,
    # --- word2vec model parameters
    vector_size = 50, # este parâmetro é o equivalente ao número de features. 
    min_count = 2, workers = 2,
    # --- vector_combination
    vector_combination = lambda x: np.mean(x, axis = 0),
)

In [23]:
X_trains['word2vec_sum'][:2, :]

array([[-0.06305836, -0.20200952, -0.09266718,  0.20211539, -0.3365239 ,
        -0.67118025,  0.91174768,  1.29723158, -1.2561081 , -0.26237645,
        -0.46331651, -0.8443768 ,  0.09114449,  0.2859693 , -0.59503645,
         0.19414936,  0.60102654,  0.25231675, -1.09676856, -0.74585192,
         0.30364978,  0.63270831,  0.96508832, -0.3080457 ,  0.44224393,
         0.26311895, -0.75946613, -0.05639867, -0.9584822 ,  0.0464241 ,
         0.33235208,  0.12054321, -0.07041632,  0.20535993, -0.48483081,
         0.59237931,  0.55151641,  0.14408598,  0.21051726, -0.43548177,
         0.76515405, -0.38607587, -0.20458422, -0.07331788,  1.31086324,
         0.2223587 , -0.17039777, -0.68101051,  0.69712637,  0.25459542],
       [-0.00804764, -0.05777715, -0.08113232,  0.08375977, -0.1659744 ,
        -0.27797719,  0.39942197,  0.59975064, -0.51845135, -0.13187836,
        -0.2138146 , -0.34640594,  0.03107198,  0.12967061, -0.34072803,
         0.13442201,  0.24537586,  0.13270103, -0.

### *Doc2Vec*

O *Doc2Vec* ([Gensim](https://radimrehurek.com/gensim/auto_examples/tutorials/run_doc2vec_lee.html)) é um modelo similar ao *Word2Vec*, mas que leva em consideração também o contexto de cada frase na construção dos vetores de similaridade.

In [24]:
from gensim.models import doc2vec

# função para ler o corpus e tagear os documentos (no caso, tweets)
def read_corpus(list_sentences, tokens_only = False):
    if tokens_only:
        return list_sentences
    else:
        # For training data, add tags
        lista = []
        for i, line in enumerate(list_sentences):
            lista.append(doc2vec.TaggedDocument(line, [i]))

        return lista
    
train_corpus = read_corpus(X_train_tokens)
test_corpus = read_corpus(X_test_tokens, tokens_only = True)

d2v_model = doc2vec.Doc2Vec(vector_size = 50, min_count = 2, epochs = 20)

d2v_model.build_vocab(train_corpus)

d2v_model.train(
    train_corpus, 
    total_examples = d2v_model.corpus_count, 
    epochs = d2v_model.epochs)

In [25]:
# exemplo: vetor de uma frase contendo duas palavras: 'bolsonaro' e 'haddad'

d2v_model.infer_vector(['bolsonaro', 'haddad'])

array([ 0.01400302, -0.02584669,  0.00031836, -0.00150514, -0.01399519,
       -0.02254681,  0.03149108,  0.03564623, -0.06478699, -0.03257909,
       -0.03688202, -0.02355443,  0.00209444, -0.00101186, -0.03051149,
        0.02547154,  0.01096833, -0.00045831, -0.03104848, -0.02308318,
        0.00204974,  0.03852638,  0.04322977, -0.02161267,  0.03768882,
        0.02182475, -0.04520045,  0.00242103, -0.04459028,  0.01308559,
        0.01355772,  0.01562951, -0.00284459,  0.04227247, -0.02978106,
        0.04166187,  0.0138301 ,  0.00326481,  0.02070507, -0.02589211,
        0.02491639,  0.01667134,  0.00890227, -0.00474655,  0.03985274,
        0.0103071 , -0.01104184, -0.0334091 ,  0.03692046, -0.00392407],
      dtype=float32)

In [26]:
# função para, dado um modelo doc2vec e um conjunto de tokens, construir o vetor associado
def build_doc2vec_vector(d2v_model, phrases):
    X = []

    for phrase in phrases:
        vecs = []
        vecs.append(d2v_model.infer_vector(phrase))
        
        X.append(vecs)
        
    X_d2v = np.array(X)[:, 0, :]

    return X_d2v

# função para, dados conjuntos de frases de treino e teste, construir os vetores
# associados

def build_doc2vec_model(
    X_train, X_test, 
    is_token = False,
    **kwargs
):

    if is_token:
        X_train_tokens = X_train
        X_test_tokens = X_test
    else:
        X_train_tokens = X_train.str.split(' ').to_list()
        X_test_tokens = X_test.str.split(' ').to_list()
    
    # make corpus
    train_corpus = read_corpus(X_train_tokens)
    test_corpus = read_corpus(X_test_tokens, tokens_only = True)

    # instantiate doc2vec model
    d2v_model = doc2vec.Doc2Vec(**kwargs)

    # build vocabulary
    d2v_model.build_vocab(train_corpus)

    # train model
    d2v_model.train(
        train_corpus, 
        total_examples = d2v_model.corpus_count, 
        epochs = d2v_model.epochs
    )

    # build vectors
    X_train_d2v = build_doc2vec_vector(
        d2v_model = d2v_model, 
        phrases = X_train_tokens
    )
    X_test_d2v = build_doc2vec_vector(
        d2v_model = d2v_model, 
        phrases = X_test_tokens
    )

    return d2v_model, X_train_d2v, X_test_d2v

In [27]:
d2v_model, X_trains['doc2vec'], X_tests['doc2vec'] = build_doc2vec_model(
    X_train, X_test, 
    is_token = False,
    vector_size = 50, min_count = 2, epochs = 20,  # doc2vec model arguments
)

In [30]:
X_trains['doc2vec'][:2, :]

array([[-0.01307306, -0.00381729,  0.01335488,  0.01585352,  0.00773212,
         0.00179032, -0.00681294,  0.00483858, -0.01388805,  0.0173075 ,
         0.0064855 ,  0.01094348,  0.0075306 ,  0.00943774,  0.0010797 ,
         0.00032177,  0.00946476,  0.00330576, -0.00337737, -0.00676753,
        -0.02491206, -0.00238715,  0.01878353,  0.00761508, -0.00095866,
        -0.00463589,  0.01800724, -0.00082818,  0.00514946, -0.01250828,
        -0.00029735, -0.0079055 , -0.00708129, -0.01498468, -0.0102274 ,
         0.00218874, -0.0064918 , -0.00637636,  0.0160045 , -0.00766212,
        -0.01217097, -0.00655804, -0.00773143, -0.01970037,  0.00555086,
        -0.01224645,  0.00232145, -0.01259743, -0.00470705, -0.00246265],
       [ 0.0080996 ,  0.0099153 ,  0.00129044, -0.00274199,  0.0016918 ,
         0.01853403,  0.00152807, -0.00361797,  0.0172362 ,  0.0057939 ,
         0.01489335,  0.00026075, -0.00326554, -0.00418042,  0.00342851,
         0.00361359,  0.00770666, -0.00032519,  0.

## Em resumo

Modelos de aprendizado de máquina não conseguem trabalhar com texto, somente números. Temos que ter algumas técnicas para transformar o texto em números.

Treinamos quatro modelos que transformam texto puro em vetores de *features* com os quais os modelos de aprendizado de máquina conseguem trabalhar:

* *Bag of Words* puro, ou `CountVectorizer`;
* *Bag of Words* TF-IDF (ou seja, considerando a influência de cada *tweet*);
* *Word2vec* (com duas maneiras de combinar os vetores de cada palavra); e
* *Doc2vec*

In [29]:
print(X_trains.keys())

dict_keys(['bow', 'tfidf', 'word2vec_sum', 'word2vec_mean', 'doc2vec'])


O próximo passo é treinar os modelos de aprendizado de máquina para prever, dado um *tweet* novo, qual o tom (sentimento) dele: positivo, negativo ou neutro.